In [10]:
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8.tgz
!tar -xvf spark-2.4.8.tgz

--2022-07-04 21:22:49--  https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15829241 (15M) [application/x-gzip]
Saving to: ‘spark-2.4.8.tgz.1’

spark-2.4.8.tgz.1   100%[===================>]  15.10M  1.97MB/s    in 15s     

2022-07-04 21:23:05 (1.00 MB/s) - ‘spark-2.4.8.tgz.1’ saved [15829241/15829241]

spark-2.4.8/
spark-2.4.8/.gitattributes
spark-2.4.8/.github/
spark-2.4.8/.github/PULL_REQUEST_TEMPLATE
spark-2.4.8/.github/workflows/
spark-2.4.8/.github/workflows/build_and_test.yml
spark-2.4.8/.github/workflows/publish_snapshot.yml
spark-2.4.8/.github/workflows/test_report.yml
spark-2.4.8/CONTRIBUTING.md
spark-2.4.8/LICENSE
spark-2.4.8/NOTICE
spark-2.4.8/R/
spark-2.4.8/R/.gitignore
spark-2.4.8/R/CRAN_RELEASE.md
spa

In [5]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

--2022-07-04 21:07:30--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.24.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.24.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12002039 (11M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar’

spark-streaming-kaf 100%[===================>]  11.45M  4.38MB/s    in 2.6s    

2022-07-04 21:07:33 (4.38 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar’ saved [12002039/12002039]



Spark Streaming

In [2]:
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "./spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars ./spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [3]:
import findspark
findspark.init()

In [10]:
import pyspark
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json
import pandas as pd

In [37]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def Sentiment(sentence):
    sidObj = SentimentIntensityAnalyzer()
    sentimentDict = sidObj.polarity_scores(sentence)
    sentimentValue = ""
    if sentimentDict['compound'] >= 0.05 :
        sentimentValue = "Positive"
    elif sentimentDict['compound'] <= -0.05 :
        sentimentValue = "Negative"
    else:
        sentimentValue = "Neutrual"
    d = {
        "Sentence": sentence,
        "Sentiment": sentimentValue
    }
    with open("data.json", "a") as out: 
        json.dump(d,out)
        out.write("\n")
        out.close()
    return d

In [48]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,5)

kafka_topic_name = "twitter"
kafka_bootstrap_servers = '127.0.0.1:9092'

kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
lines = kvs.map(lambda x: json.loads(x[1]))
out = lines.map(lambda line: Sentiment(line['text']))
out.pprint()


ssc.start()
# stream will run for 30 sec
ssc.awaitTerminationOrTimeout(40)
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2022-07-05 23:26:40
-------------------------------------------



-------------------------------------------
Time: 2022-07-05 23:26:45
-------------------------------------------
{'Sentence': '@elonmusk Fix my twitter problem elon musk when i open my account instead of larry black circle open https://t.co/wZaTy52Oeh', 'Sentiment': 'Negative'}
{'Sentence': '@PolachekRachael @elonmusk It berra be auntie Rachael 😑😂😂😂🐶💞', 'Sentiment': 'Positive'}
{'Sentence': '.@elonmusk, Is there inherent order in nature or is it all chaos and chance?', 'Sentiment': 'Negative'}
{'Sentence': '@TheDailyEdge @elonmusk @ruthbenghiat I hope Elon comes around.', 'Sentiment': 'Positive'}
{'Sentence': '@WholeMarsBlog @DirtyTesLa @elonmusk Number 45s notecards after the Marjorie Stoneman Douglass mass shooting. \n\nBoth sides seem to depend on a scripted stationery. https://t.co/v7g9WmRGVR', 'Sentiment': 'Positive'}
{'Sentence': "@elonmusk @jhkchan Gm 👋 That's not a joke, and definitely not fake. Just a unique process that you've probably never seen before, bringing targeted us

-------------------------------------------
Time: 2022-07-05 23:26:50
-------------------------------------------
{'Sentence': "@EmmetPeppers @elonmusk That tweet wasn't interesting because of real price of gas, it was interesting because of the closing price of TSLA that day.", 'Sentiment': 'Positive'}
{'Sentence': "This was destined to happen all of it @elonmusk whether I'm alive or dead", 'Sentiment': 'Negative'}
{'Sentence': '@KDORR_USA @TwitterSupport Maybe if @elonmusk owned it something would come of it. Right now, every Twitter employee is having a melt down and cheering on the fake news lib politicians spewing their garbage.', 'Sentiment': 'Positive'}
{'Sentence': 'The @homer_coin Presale is coming! you have to check it out guys, it;s probably the biggest project of this summer, I see a big hype on it LFG!!! #BTC #BSC #ETH #BSCgems #Launch #Presale #Shib #Doge #Elon @elonmusk #crypto #homercoin #USDT #BNB #pump #Bitcoin #NFTs #metavers #p2e', 'Sentiment': 'Neutrual'}
{'Sentenc

-------------------------------------------
Time: 2022-07-05 23:26:55
-------------------------------------------
{'Sentence': '@adventuregirl @TimRunsHisMouth @wasif_localseeo You have 1.1 million followers, and can’t get more than 10 likes in 25 minutes? How much did you pay for that bot army?\nCc: @elonmusk', 'Sentiment': 'Positive'}
{'Sentence': '@GodHatesNFTees @elonmusk Very valid point 🤔', 'Sentiment': 'Neutrual'}
{'Sentence': "@figjamben @chan_lambda @sachie2021 @dailystar @elonmusk Its not his money. It's the his daddies and the American tax payer", 'Sentiment': 'Neutrual'}
{'Sentence': '@GamingShibaBSC @elonmusk Very excited to participate\n\nJoin now for participate!\n@CalonSultan69 \n@vhia_ovie  \n@Ujang_Geol \n@sans_sanjaya \n@yayuyuhana2', 'Sentiment': 'Positive'}
{'Sentence': '@KrestTest @WholeMarsBlog @elonmusk FSD subscription costs $200 though…how would EAP cost more?', 'Sentiment': 'Neutrual'}
{'Sentence': '@RealAndyLeeShow This is extremely scary. @elonmusk', 'Senti

-------------------------------------------
Time: 2022-07-05 23:27:00
-------------------------------------------
{'Sentence': '@fancyeagledance @rpetty @RealSaavedra @elonmusk @Twitter I know it’s definitely illegal to make such threats, but likely still freedom of speech (both in principle and in law).', 'Sentiment': 'Positive'}
{'Sentence': 'What do you think @elonmusk https://t.co/XtKym9y9LQ', 'Sentiment': 'Neutrual'}
{'Sentence': "@libsoftiktok @FBI @elonmusk Why isn't @Twitter removing these tweets or at least putting a warning on them or banning the accounts?", 'Sentiment': 'Negative'}
{'Sentence': '@hippo_inu @ChinaPumpWXC @elonmusk @cz_binance @dogecoin @BabyDogeCoin @kucoincom @Bybit_Official @Hotbit_news @pinkecosystem @binance Nice project and congratulations to the team for their efforts and dedication and highly appreciated the visionary thought of the projector and it will create history\n\n@Udiiiinnnnnn \n@Izzy21_05 \n@Abcalenaa', 'Sentiment': 'Positive'}
{'Sentence': '

-------------------------------------------
Time: 2022-07-05 23:27:05
-------------------------------------------
{'Sentence': '@elonmusk another thing I havent had 1 second to myself in 5years yourfoxcatchers are constantly communicating with me even when Im havingsex I can hear you guys saying stuff either trying to stop me to keep using my subconcious mind or to try and get me to be bisexual which Icant Momo🚫🤖', 'Sentiment': 'Negative'}
{'Sentence': '@broman6900 @DA2Z5 @itsmyke @Agirlhasnofame @elonmusk You can think whatever you want about me, just like I can think that you definitely don’t know how to work smarter and think that people should just be as basic as you are. I don’t need to exchange my life for work to live well… you do. That’s the difference between us', 'Sentiment': 'Positive'}
{'Sentence': '@cnunezimages @elonmusk @SpaceX @SpaceIntellige3 Phenomenal!! ❤️', 'Sentiment': 'Neutrual'}
{'Sentence': '@EliErlick @FBI @Twitter @elonmusk Handle this shit.', 'Sentiment': 'Ne

-------------------------------------------
Time: 2022-07-05 23:27:10
-------------------------------------------
{'Sentence': '@frztoken @elonmusk Hey ELON, give up dogs and cats and be strong with this alliance and be very beautiful and powerful with this project 💸👑🦁🎯🔥🚀🚀🌏⤴️🆙🌓↗️\U0001fa90☄️\n#FRZSS_Token #FRZSS_Solar_System #FRZSS #Frzss_renewable_energy #FRZSS1000x\n\n💥💥💥💥 https://t.co/bRzRcrD2Cy', 'Sentiment': 'Positive'}
{'Sentence': '@elonmusk Need that beta please!', 'Sentiment': 'Positive'}
{'Sentence': '@elonmusk Yourmotherfuckingright 🙂\U0001fae1🦋🐺⚡️⚡️🕊🐾🌹🇺🇸✈️🇺🇦\U0001fa96🍻🌮🎯🐊🌪🌒🌑🌑🌑🌑🌑🌑🌑🌑🌑🌑🌘🌗🌓🌖🌙🌖🌔🌑🌪🐊🎯🐺⚡️⚡️🕊slava Ukraine', 'Sentiment': 'Positive'}
{'Sentence': '@elonmusk Tesla okay I see, how much is it to replace barriers if I buy a used Tesla? Where do the old batteries go?', 'Sentiment': 'Positive'}
{'Sentence': '@laurenweinstein Power corrupts, absolute power corrupts absolutely.\n\n@elonmusk', 'Sentiment': 'Neutrual'}
{'Sentence': '@elonmusk I was in Las Vegas two weeks ago an

-------------------------------------------
Time: 2022-07-05 23:27:15
-------------------------------------------
{'Sentence': '@elonmusk Since you said Tesla would pay for employees to travel to another state for an abortion if needed, will you offer financial aid to employees that want to adopt children?', 'Sentiment': 'Positive'}
{'Sentence': "@elonmusk It's a good project and I hope this project will gain more popularity in the future. 🚀 🚀 \n@posschain", 'Sentiment': 'Positive'}
{'Sentence': '@felipe_molag @elonmusk Me too', 'Sentiment': 'Neutrual'}
{'Sentence': '#CryptoMarket #LUNACLASSIC #lunatics #lunaburn we will rise to The moon when the burn is a fact.\n@elonmusk @binance @bitvavocom  we hope for Total suport from you.\nLet a #whale give the push that the community diserve. @stablekwon', 'Sentiment': 'Positive'}
{'Sentence': 'Maybe #dogecoin is better than #Bitcoin \n@elonmusk https://t.co/i8QRGAD0Gh', 'Sentiment': 'Positive'}
{'Sentence': '@elonmusk Uh "real" cheese duh', 'S

In [41]:
ssc.stop()
sc.stop()